In [1]:
# Load Packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px

import warnings

warnings.filterwarnings("ignore")

from typing import List

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

import keras
import time

rootpath = ".."
import model_prep


step_back = 6  # window size = 6*5 = 30 mins
season_map = {
    "spring": [3, 4, 5],
    "summer": [6, 7, 8],
    "fall": [9, 10, 11],
    "winter": [12, 1, 2],
}

2023-10-03 17:48:55.397837: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
building_name="ESB"
tower_number=1
season="summer"
features = ['ESB_Tower_1 enteringWaterTemp', 'ESB_Tower_1 outdoorAirDryBulb', 'ESB_Tower_1 outdoorAirWetBulb', 'ESB_Tower_1 vfdPercent', 'ESB_Tower_1 fanA_vfdPower', 'ESB_Tower_1 fanB_vfdPower', 'ESB_Tower_1 dayOfWeek', 'ESB_Tower_1 hourOfDay', 'ESB_Tower_1 efficiency']
target = 'ESB_Tower_1 leavingWaterTemp'
train_percentage = 0.75
use_delta = True
shuffle_seed = 42

In [4]:
"""
1. Convert data into a model-compatible shape
"""

lstm_df, first_temp = model_prep.create_preprocessed_lstm_df(
    building_name=building_name,
    tower_number=tower_number,
    features=features,
    target=target,
    season=season,
    use_delta=use_delta,
)
if not season:
    season = "allyear"

"""
2. Split data into training and testing sets
"""

X = lstm_df.drop(f"{target}(t)", axis=1)  # drop target column
y = lstm_df[f"{target}(t)"]  # only have target column

# split into input and outputs
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=(1 - train_percentage), shuffle=True, random_state=shuffle_seed
)

# scale feature data
scaler = MinMaxScaler().fit(X_train)
X_train[X_train.columns] = scaler.transform(X_train)
X_test[X_test.columns] = scaler.transform(X_test)

"""
3. Get timestepped data as a 3D vector
"""
vec_X_train = model_prep.df_to_3d(
    lstm_dtframe=X_train, num_columns=len(features) + 1, step_back=step_back
)
vec_X_test = model_prep.df_to_3d(
    lstm_dtframe=X_test, num_columns=len(features) + 1, step_back=step_back
)

vec_y_train = y_train.values
vec_y_test = y_test.values

# print(vec_X_train.shape, vec_X_test.shape, vec_y_train.shape, vec_y_test.shape)

In [13]:
"""
4. Create and Train model
"""
dropout_rate = 0.0
weight_constraint = keras.constraints.MaxNorm(2.0)
# lstmcells = ?
activation = "tanh"
optimizer = "Adamax"

model = keras.models.Sequential()
model.add(
    keras.layers.GRU(
        32,
        activation=activation,
        return_sequences=True,
        input_shape=(vec_X_train.shape[1], vec_X_train.shape[2]),
        kernel_constraint=weight_constraint,
        recurrent_dropout=dropout_rate,
    )
)

model.add(keras.layers.Dense(1))
model.compile(loss="mse", optimizer=optimizer)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 6, 32)             5504      
                                                                 
 dense_3 (Dense)             (None, 6, 1)              33        
                                                                 
Total params: 5,537
Trainable params: 5,537
Non-trainable params: 0
_________________________________________________________________


In [14]:
start_time = time.time()
history = model.fit(
    vec_X_train,
    vec_y_train,
    epochs=50,
    batch_size=72,
    validation_data=(vec_X_test, vec_y_test),
    verbose=0,
    shuffle=False,
)
end_time = time.time()
training_time = end_time - start_time


# make predictions
yhat = model.predict(vec_X_test)

40/40 [==============================] - 0s 1ms/step


In [15]:
yhat

array([[[9.143715 ],
        [8.795067 ],
        [8.803518 ],
        [8.872602 ],
        [8.857102 ],
        [8.880585 ]],

       [[7.99538  ],
        [8.592178 ],
        [8.462436 ],
        [8.539933 ],
        [8.516652 ],
        [8.518467 ]],

       [[9.553431 ],
        [8.886969 ],
        [8.989593 ],
        [8.949055 ],
        [9.0090065],
        [8.970984 ]],

       ...,

       [[8.899769 ],
        [8.658319 ],
        [8.728067 ],
        [8.781568 ],
        [8.741146 ],
        [8.750918 ]],

       [[9.943085 ],
        [9.059908 ],
        [9.184248 ],
        [9.235409 ],
        [9.226103 ],
        [9.234907 ]],

       [[4.766969 ],
        [7.1742544],
        [7.4852643],
        [7.7333183],
        [7.6961474],
        [7.9363894]]], dtype=float32)

In [23]:
yhat[:, 0, 0].shape

(1255,)

In [18]:
vec_y_test.reshape((vec_y_test.shape[0])).shape

(1255,)